# Deep Space Telemetry Budget Monitor

This notebook tracks and visualizes budget utilization for the Deep Space Telemetry operational category. It provides:
- Real-time cost monitoring
- Usage trend analysis  
- Resource utilization breakdowns
- Budget threshold alerts
- Spending forecasts

**Category**: Deep Space Telemetry  
**Monthly Budget**: $10,000  
**Last Updated**: April 2025

In [0]:
# Category Configuration

CATEGORY_NAME = "DEEP_SPACE_TELEMETRY"
MONTHLY_BUDGET = 10000
ALERT_THRESHOLDS = [75, 90, 100]

# Date range for analysis
import datetime
current_date = datetime.datetime.now()
first_day_of_month = current_date.replace(day=1)
last_day_of_month = (first_day_of_month + datetime.timedelta(days=32)).replace(day=1) - datetime.timedelta(days=1)

DATE_RANGE = {
    "start": first_day_of_month.strftime("%Y-%m-%d"),
    "end": last_day_of_month.strftime("%Y-%m-%d")
}

# Visualization settings
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = [12, 8]

In [0]:
# Imports and setup
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, datediff, current_date

# Initialize Spark session
spark = SparkSession.builder.appName(f"{CATEGORY_NAME}_BudgetMonitor").getOrCreate()

In [0]:
# Data retrieval functions

def get_category_usage(category_name, date_range):
    """Retrieve usage data for the specific operational category."""
    query = f"""
    SELECT 
        usage_date,
        usage_unit,
        usage_quantity,
        list_price,
        usage_quantity * list_price as total_cost,
        sku_name,
        usage_metadata.cluster_id,
        usage_metadata.warehouse_id,
        tags.Project,
        tags.Owner
    FROM system.billing.usage
    WHERE tags.OperationalCategory = '{category_name}'
    AND usage_date BETWEEN '{date_range['start']}' AND '{date_range['end']}'
    ORDER BY usage_date ASC
    """
    return spark.sql(query)

def get_daily_costs(category_name, date_range):
    """Get daily cost summary."""
    query = f"""
    SELECT 
        usage_date,
        SUM(usage_quantity * list_price) as daily_cost
    FROM system.billing.usage
    WHERE tags.OperationalCategory = '{category_name}'
    AND usage_date BETWEEN '{date_range['start']}' AND '{date_range['end']}'
    GROUP BY usage_date
    ORDER BY usage_date ASC
    """
    return spark.sql(query)

def get_resource_breakdown(category_name, date_range):
    """Get cost breakdown by resource type."""
    query = f"""
    SELECT 
        sku_name,
        SUM(usage_quantity * list_price) as total_cost,
        COUNT(DISTINCT usage_date) as days_used
    FROM system.billing.usage
    WHERE tags.OperationalCategory = '{category_name}'
    AND usage_date BETWEEN '{date_range['start']}' AND '{date_range['end']}'
    GROUP BY sku_name
    ORDER BY total_cost DESC
    """
    return spark.sql(query)

def get_user_spending(category_name, date_range):
    """Get spending by user."""
    query = f"""
    SELECT 
        tags.Owner as user_email,
        SUM(usage_quantity * list_price) as total_cost,
        COUNT(DISTINCT usage_date) as active_days
    FROM system.billing.usage
    WHERE tags.OperationalCategory = '{category_name}'
    AND tags.Owner IS NOT NULL
    AND usage_date BETWEEN '{date_range['start']}' AND '{date_range['end']}'
    GROUP BY tags.Owner
    ORDER BY total_cost DESC
    """
    return spark.sql(query)

In [0]:
# Retrieve data for analysis
usage_data = get_category_usage(CATEGORY_NAME, DATE_RANGE)
daily_costs = get_daily_costs(CATEGORY_NAME, DATE_RANGE).toPandas()
resource_breakdown = get_resource_breakdown(CATEGORY_NAME, DATE_RANGE).toPandas()
user_spending = get_user_spending(CATEGORY_NAME, DATE_RANGE).toPandas()

# Calculate total spending for the month
total_spent = daily_costs['daily_cost'].sum() if not daily_costs.empty else 0
budget_utilization = (total_spent / MONTHLY_BUDGET) * 100

# Calculate days elapsed and remaining
days_elapsed = (current_date - first_day_of_month).days + 1
days_in_month = (last_day_of_month - first_day_of_month).days + 1
days_remaining = days_in_month - days_elapsed

print(f"Deep Space Telemetry Budget Status:")
print(f"Total Spent: ${total_spent:,.2f}")
print(f"Monthly Budget: ${MONTHLY_BUDGET:,.2f}")
print(f"Budget Utilization: {budget_utilization:.1f}%")
print(f"Days Elapsed: {days_elapsed} of {days_in_month}")

In [0]:
# Time-series spending visualization

# Create cumulative spending
if not daily_costs.empty:
    daily_costs['cumulative_cost'] = daily_costs['daily_cost'].cumsum()
    
    # Create the figure
    fig = go.Figure()
    
    # Add daily spending bar chart
    fig.add_trace(go.Bar(
        x=daily_costs['usage_date'],
        y=daily_costs['daily_cost'],
        name='Daily Spending',
        marker_color='lightblue'
    ))
    
    # Add cumulative spending line
    fig.add_trace(go.Scatter(
        x=daily_costs['usage_date'],
        y=daily_costs['cumulative_cost'],
        name='Cumulative Spending',
        line=dict(color='navy', width=3)
    ))
    
    # Add budget threshold lines
    for threshold in ALERT_THRESHOLDS:
        threshold_value = MONTHLY_BUDGET * (threshold / 100)
        fig.add_hline(
            y=threshold_value,
            line_dash="dash",
            line_color="red" if threshold == 100 else "orange" if threshold == 90 else "yellow",
            annotation_text=f"{threshold}% Threshold",
            annotation_position="right"
        )
    
    fig.update_layout(
        title='Deep Space Telemetry: Daily Spending and Cumulative Budget',
        xaxis_title='Date',
        yaxis_title='Cost (USD)',
        height=600,
        showlegend=True
    )
    
    fig.show()
else:
    print("No spending data available for the selected period.")

In [0]:
# Resource utilization pie chart

if not resource_breakdown.empty:
    fig = px.pie(
        resource_breakdown,
        values='total_cost',
        names='sku_name',
        title='Cost Distribution by Resource Type',
        hole=0.4
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label'
    )
    
    fig.update_layout(height=600)
    fig.show()
else:
    print("No resource breakdown data available.")

In [0]:
# User spending analysis

if not user_spending.empty:
    fig = px.bar(
        user_spending.head(10),  # Top 10 users
        x='user_email',
        y='total_cost',
        title='Top 10 Users by Spending',
        labels={'user_email': 'User', 'total_cost': 'Total Cost (USD)'},
        color='total_cost',
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(
        xaxis_tickangle=-45,
        height=600
    )
    
    fig.show()
else:
    print("No user spending data available.")

In [0]:
# Budget gauge chart

fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=budget_utilization,
    title={'text': f"Budget Utilization (${total_spent:,.2f} of ${MONTHLY_BUDGET:,.2f})"},
    delta={'reference': 75, 'increasing': {'color': "red"}},
    gauge={
        'axis': {'range': [0, 100], 'tickwidth': 1},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, 75], 'color': "lightgreen"},
            {'range': [75, 90], 'color': "yellow"},
            {'range': [90, 100], 'color': "red"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 100
        }
    }
))

fig.update_layout(
    height=400,
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.show()

In [0]:
# Spending forecast

if not daily_costs.empty and len(daily_costs) > 1:
    # Calculate average daily spend
    avg_daily_spend = total_spent / days_elapsed
    
    # Project remaining spending
    projected_total = total_spent + (avg_daily_spend * days_remaining)
    projected_utilization = (projected_total / MONTHLY_BUDGET) * 100
    
    # Create forecast data
    forecast_dates = pd.date_range(current_date.date() + datetime.timedelta(days=1), last_day_of_month)
    forecast_costs = [total_spent + (avg_daily_spend * i) for i in range(1, len(forecast_dates) + 1)]
    
    fig = go.Figure()
    
    # Actual spending
    fig.add_trace(go.Scatter(
        x=daily_costs['usage_date'],
        y=daily_costs['cumulative_cost'],
        name='Actual Spending',
        line=dict(color='blue', width=3)
    ))
    
    # Forecasted spending
    fig.add_trace(go.Scatter(
        x=forecast_dates,
        y=forecast_costs,
        name='Projected Spending',
        line=dict(color='red', width=3, dash='dash')
    ))
    
    # Budget line
    fig.add_hline(
        y=MONTHLY_BUDGET,
        line_dash="solid",
        line_color="green",
        annotation_text="Monthly Budget",
        annotation_position="right"
    )
    
    fig.update_layout(
        title='Spending Forecast: Deep Space Telemetry',
        xaxis_title='Date',
        yaxis_title='Cumulative Cost (USD)',
        height=600
    )
    
    fig.show()
    
    print(f"\nForecast Summary:")
    print(f"Average Daily Spend: ${avg_daily_spend:.2f}")
    print(f"Projected Month-end Total: ${projected_total:.2f}")
    print(f"Projected Budget Utilization: {projected_utilization:.1f}%")
    
    if projected_utilization > 100:
        overspend = projected_total - MONTHLY_BUDGET
        print(f"WARNING: Projected overspend of ${overspend:.2f}")
else:
    print("Insufficient data for forecast analysis.")

In [0]:
# Alert status and recommendations

def generate_alert_status():
    """Generate alert status based on current budget utilization."""
    alerts = []
    
    for threshold in ALERT_THRESHOLDS:
        if budget_utilization >= threshold:
            severity = "CRITICAL" if threshold == 100 else "WARNING" if threshold == 90 else "NOTICE"
            alerts.append({
                "threshold": threshold,
                "severity": severity,
                "message": f"Budget utilization has reached {budget_utilization:.1f}% (threshold: {threshold}%)",
                "action_required": "Immediate spending controls recommended" if threshold >= 90 else "Monitor closely"
            })
    
    return alerts

def generate_recommendations():
    """Generate cost optimization recommendations."""
    recommendations = []
    
    # Calculate spending rate
    if days_elapsed > 0:
        spending_rate = total_spent / days_elapsed
        ideal_rate = MONTHLY_BUDGET / days_in_month
        
        if spending_rate > ideal_rate * 1.2:
            recommendations.append({
                "category": "Spending Rate",
                "issue": f"Current spending rate (${spending_rate:.2f}/day) exceeds ideal rate (${ideal_rate:.2f}/day)",
                "recommendation": "Reduce daily usage or optimize resource selection"
            })
    
    # Analyze top resource usage
    if not resource_breakdown.empty:
        top_resource = resource_breakdown.iloc[0]
        if top_resource['total_cost'] > total_spent * 0.4:
            recommendations.append({
                "category": "Resource Concentration",
                "issue": f"{top_resource['sku_name']} accounts for {top_resource['total_cost']/total_spent*100:.1f}% of spending",
                "recommendation": "Consider alternative resource types or optimization strategies"
            })
    
    # Analyze user concentration
    if not user_spending.empty:
        top_user = user_spending.iloc[0]
        if top_user['total_cost'] > total_spent * 0.3:
            recommendations.append({
                "category": "User Concentration",
                "issue": f"Single user accounts for {top_user['total_cost']/total_spent*100:.1f}% of spending",
                "recommendation": "Review user permissions and resource allocation policies"
            })
    
    return recommendations

# Display alerts and recommendations
alerts = generate_alert_status()
recommendations = generate_recommendations()

print("\n===== ALERT STATUS =====")
if alerts:
    for alert in alerts:
        print(f"[{alert['severity']}] {alert['message']}")
        print(f"   Action: {alert['action_required']}")
else:
    print("No active alerts - budget usage is within acceptable limits.")

print("\n===== RECOMMENDATIONS =====")
if recommendations:
    for rec in recommendations:
        print(f"• {rec['category']}: {rec['issue']}")
        print(f"  → {rec['recommendation']}")
else:
    print("No specific recommendations at this time.")

In [0]:
# Export summary data

def export_budget_summary():
    """Export budget summary as a CSV file."""
    summary_data = {
        "Category": CATEGORY_NAME,
        "Month": first_day_of_month.strftime("%B %Y"),
        "Total Budget": MONTHLY_BUDGET,
        "Total Spent": total_spent,
        "Budget Utilization %": budget_utilization,
        "Days Elapsed": days_elapsed,
        "Days Remaining": days_remaining,
        "Average Daily Spend": total_spent / days_elapsed if days_elapsed > 0 else 0,
        "Projected Month-end Total": total_spent + ((total_spent / days_elapsed) * days_remaining if days_elapsed > 0 else 0),
        "Alert Status": "CRITICAL" if budget_utilization >= 100 else "WARNING" if budget_utilization >= 90 else "CAUTION" if budget_utilization >= 75 else "NORMAL"
    }
    
    summary_df = pd.DataFrame([summary_data])
    
    # Export to CSV
    filename = f"deep_space_telemetry_budget_summary_{current_date.strftime('%Y%m%d')}.csv"
    summary_df.to_csv(filename, index=False)
    
    print(f"\nBudget summary exported to: {filename}")
    
    # Display summary table
    display(summary_df)

# Export the summary
export_budget_summary()

## Deep Space Telemetry Budget Monitor - Executive Summary

This notebook provides real-time monitoring and analysis of the Deep Space Telemetry operational category budget. Key features include:

1. **Daily Spending Tracking**: Time-series visualization of daily and cumulative spending
2. **Resource Analysis**: Breakdown of costs by resource type (SKU)
3. **User Attribution**: Identification of top spenders within the category
4. **Budget Alerts**: Automated threshold monitoring and alerts
5. **Forecasting**: Projection of month-end spending based on current trends
6. **Recommendations**: Automated cost optimization suggestions

### Action Items
- Review spending patterns regularly
- Address any alerts promptly
- Implement optimization recommendations where applicable
- Schedule monthly budget reviews with key stakeholders

For questions or support, contact: 